In [1]:
import json
import os
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
from scrapy.crawler import CrawlerProcess
from mantis_scrapper.mantis_scrapper.spiders.mantis_spider import MantisSpider
from bs4 import BeautifulSoup
import sys
import os

In [2]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(150000)
print(sys.getrecursionlimit())

3000
150000


In [3]:
news_webs = [
    'https://www.fiercewireless.com',
]

key_words = [
    "smartphone", "smartphone technology",
    "wearables", "smart watch", "smart glasses", "ar/vr headsets", "ar headsets", "vr headsets",
    "iphone", "android smartphone",
    "apple", "samsung", "xiaomi", "oppo", "vivo",
    "qualcomm", "nxp", "mediaTek", "cirrus logic", "skyworks", "qorvo", "broadcom", "sony", "murata",
    "wifi", "cellular", "5g", "6g", "uwb", "nfc",
    "apple pay", "android pay",
    "magsafe", "wireless charging",
    "power management", "battery management",
    "envelope tracker",
    "rf front end", "transceiver", "modem",
    "mmwave", "fr2", "fr3",
    "oled display", "uled", "ltpo", "display driver ic",
    "smartphone camera", "image sensing", "3d sensing", "camera",
    "smart audio",
    "haptics",
    "satellite to smartphone connectivity"
]

WINDOW_SIZE = "1000,600"

chrome_options = Options()
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

In [4]:
all_urls = []
scrapped_links = []
visited_links = []

In [5]:
def link_parser(web_links):
    for count, url in enumerate(web_links):
        if url.endswith('/'):
            url = url[:-1]
        if url in scrapped_links or url in visited_links:
            if len(web_links) > 1:
                print("skipping already visited: ", url)
                continue
        print("loading....", url)
        driver = webdriver.Chrome(
            executable_path=r'C:\chromedriver.exe',
            chrome_options=chrome_options
        )
        driver.get(url)
        with open("visited_urls.txt", 'a') as fb:
            fb.write(url + "\n")
            visited_links.append(url)
        html = driver.page_source

        driver.close()
        driver.quit()

        time.sleep(2)
        soup = BeautifulSoup(html, 'html.parser')
        if len(url.split('/')) > 4:
            tag_div_ids = ["article-tags-row", "article-tags-bottom-row"]
            tags_list = []
            key_word_match = False
            for tag_div_id in tag_div_ids:
                tags_divs = soup.find_all("div", {"id": tag_div_id})
                for tags_div in tags_divs:
                    tags = tags_div.find_all(
                        'a', {"class": "tag"}
                    )
                    for tag in tags:
                        if tag and tag not in tags_list:
                            tags_list.append(tag.text)
                            if tag.text.lower() in key_words:
                                key_word_match = True
            if key_word_match:
                with open("matched_urls.txt", 'a') as fa:
                    fa.write(url + " | " + ",".join(tags_list) + "\n")
                    scrapped_links.append(url)
            exit()
        urls = []
        for link in soup.find_all('a'):
            link_ = link.get('href')
            if link_:
                if not link_.startswith('https://www.'):
                    base_url = "https://" + url.split('//')[1].split('/')[0]
                    link_ = base_url + link_
                website = "https://" + link_.split('//')[1].split('/')[0]
                # print(website)
                if website not in news_webs:
                    continue
                if link_ in all_urls or link_ in scrapped_links or link_ in visited_links:
                    continue
                urls.append(link_)
                all_urls.append(link_)

        link_parser(urls)


In [6]:
if os.path.exists("matched_urls.txt"):
    with open("matched_urls.txt", 'r') as fao:
        scrapped_links = [x.split('|')[0].strip() for x in fao.readlines()]
if os.path.exists("visited_urls.txt"):
    with open("visited_urls.txt", 'r') as fbo:
        visited_links = [x.replace('\n', '').strip() for x in fbo.readlines()]
link_parser(news_webs)

loading.... https://www.fiercewireless.com


C:\Users\admin\AppData\Local\Temp\ipykernel_46584\3167443267.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(
C:\Users\admin\AppData\Local\Temp\ipykernel_46584\3167443267.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


skipping already visited:  https://www.fiercewireless.com
loading.... https://www.fiercewireless.com/private-wireless
loading.... https://www.fiercewireless.com/private-wireless/chips-are-holding-back-5g-private-wireless


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x007A6643]
	(No symbol) [0x0073BE21]
	(No symbol) [0x0063DA9D]
	(No symbol) [0x0061EF6A]
	(No symbol) [0x00693AAB]
	(No symbol) [0x006A61B6]
	(No symbol) [0x0068FB76]
	(No symbol) [0x006649C1]
	(No symbol) [0x00665E5D]
	GetHandleVerifier [0x00A1A142+2497106]
	GetHandleVerifier [0x00A485D3+2686691]
	GetHandleVerifier [0x00A4BB9C+2700460]
	GetHandleVerifier [0x00853B10+635936]
	(No symbol) [0x00744A1F]
	(No symbol) [0x0074A418]
	(No symbol) [0x0074A505]
	(No symbol) [0x0075508B]
	BaseThreadInitThunk [0x775B7D69+25]
	RtlInitializeExceptionChain [0x77D4BB9B+107]
	RtlClearBits [0x77D4BB1F+191]


In [ ]:
with open('all_urls.txt', 'w') as f:
    f.write("\n".join(all_urls))